In [1]:
import pandas as pd 
#import geopandas as gp
import numpy as np
#import altair as alt 
#import gpdvega 
import json
import matplotlib.pyplot as plt
from ast import literal_eval
#alt.renderers.enable('notebook') # render for Jupyter Notebook

In [27]:
TURNSTILE_DATA = '/nas/dft/ire/jujiang/DataClinic2019Q4HackDay/data/hourly_turnstile.csv.gz'
OUTAGE_DATA = '/nas/dft/ire/jujiang/DataClinic2019Q4HackDay/data/hourly_outage.csv'
TURNSTILE_EQUIPMENT_DATA = '/nas/dft/ire/kmohan/hackday_2019_q4_mta_accessibility/data/ee_turnstile_join.csv'
OUTPUT = '/nas/dft/ire/jujiang/DataClinic2019Q4HackDay/data/hourly_turnstile_with_outage.csv.gz'

In [5]:
turnstile = pd.read_csv(TURNSTILE_DATA)
outage = pd.read_csv(OUTAGE_DATA)

/nas/dft/ire/jujiang/.conda/envs/mta_env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (2,3,4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
subway_turnstile = pd.read_csv(TURNSTILE_EQUIPMENT_DATA,converters={"remote": literal_eval})
lst_col = 'remote'
subway_turnstile = pd.DataFrame({col:np.repeat(subway_turnstile[col].values, subway_turnstile[lst_col].str.len())
              for col in subway_turnstile.columns.difference([lst_col])}).assign(**{lst_col:np.concatenate(subway_turnstile[lst_col].values)})[subway_turnstile.columns.tolist()]
subway_turnstile.drop(columns = ['Unnamed: 0'], inplace=True)
subway_turnstile.set_index(['station_name', 'equipment_id'], inplace=True)

In [11]:
outages = pd.read_csv(OUTAGE_DATA)
outage_with_remote = outages.join(subway_turnstile, on=['Station', 'Equipment number'])
outage_with_remote.head()
outage_with_remote.rename(columns={"Time":"datetime", "remote": "UNIT"}, inplace=True)
outage_with_remote.datetime = pd.to_datetime(outage_with_remote.datetime)
outage_with_remote.drop(columns=['Unnamed: 0'])
outage_with_remote.set_index(['datetime', 'UNIT'], inplace=True)

In [14]:
turnstile.columns

Index(['Unnamed: 0', 'datetime', 'LINENAME', 'STATION', 'datetime2', 'UNIT',
       'SCP', 'ENTRIES', 'EXITS', 'entry_diff', 'exit_diff', 'entry_diff_abs',
       'exit_diff_abs', 'time_diffs'],
      dtype='object')

In [15]:
turnstile.head()

,Unnamed: 0,datetime,LINENAME,STATION,datetime2,UNIT,SCP,ENTRIES,EXITS,entry_diff,exit_diff,entry_diff_abs,exit_diff_abs,time_diffs
0,0,2019-01-01 04:00:00,1,103 ST,2019-01-01 04:00:00,R191,00-00-00,1640806.0,659187.0,117.0,62.0,117.0,62.0,4.0
1,1,2019-01-01 05:00:00,1,103 ST,2019-01-01 04:00:00,R191,00-00-00,1640806.0,659187.0,98.0,58.0,117.0,62.0,4.0
2,2,2019-01-01 06:00:00,1,103 ST,2019-01-01 04:00:00,R191,00-00-00,1640806.0,659187.0,79.0,54.0,117.0,62.0,4.0
3,3,2019-01-01 07:00:00,1,103 ST,2019-01-01 04:00:00,R191,00-00-00,1640806.0,659187.0,60.0,50.0,117.0,62.0,4.0
4,4,2019-01-01 08:00:00,1,103 ST,2019-01-01 08:00:00,R191,00-00-00,1640847.0,659233.0,41.0,46.0,41.0,46.0,4.0


In [16]:
turnstile.drop(columns=['Unnamed: 0', 'datetime2', 'ENTRIES', 'EXITS', 'time_diffs'], inplace=True)

In [21]:
turnstile['datetime'] = pd.to_datetime(turnstile['datetime'])

In [22]:
joined = turnstile.join(outage_with_remote, on=['datetime', 'UNIT'])
joined = joined[joined['Equipment type'].notna()]

In [31]:
joined.drop(columns=['Unnamed: 0'], inplace=True)
joined.to_csv(OUTPUT, compression='gzip')

In [32]:
joined.head()

,datetime,LINENAME,STATION,UNIT,SCP,entry_diff,exit_diff,entry_diff_abs,exit_diff_abs,Outage percentage,...,Planned outage,Station,Equipment type,description,borough,subway_lines,equipment_type,ada_compliant,is_active,clean_lines
93858,2019-01-03 09:00:00,1,125 ST,R034,00-00-00,309.0,105.50,70.0,30.0,0.013889,...,False,125 St,escalator,Street to Mezzanine,MN,1,ES,N,Y,1
93859,2019-01-03 10:00:00,1,125 ST,R034,00-00-00,428.5,143.25,70.0,30.0,1.000000,...,False,125 St,escalator,Street to Mezzanine,MN,1,ES,N,Y,1
93862,2019-01-03 13:00:00,1,125 ST,R034,00-00-00,465.0,175.00,548.0,181.0,0.261667,...,False,125 St,escalator,Street to Mezzanine,MN,1,ES,N,Y,1
93863,2019-01-03 14:00:00,1,125 ST,R034,00-00-00,423.5,172.00,548.0,181.0,1.000000,...,False,125 St,escalator,Street to Mezzanine,MN,1,ES,N,Y,1
93864,2019-01-03 15:00:00,1,125 ST,R034,00-00-00,382.0,169.00,382.0,169.0,0.738333,...,False,125 St,escalator,Street to Mezzanine,MN,1,ES,N,Y,1
